In [2]:
import pandas as pd
import numpy as np
import os

In [72]:
def get_diff_bin(data):
    average_bin, bin_0, bin_24 = [], [], []

    for row in range(24, data.shape[0], 25):
        bin_24.append(data.loc[row, 'bin_volume'])

    for row in range(0, data.shape[0], 25):
        bin_0.append(data.loc[row, 'bin_volume'])
        average_bin.append(data.loc[row+ 1: row+ 24, 'daily_volume'].mean())
    return average_bin, bin_0, bin_24


def transformer(data):
    transformed_nums = [-1 if num < 0 else 1 if num > 0 else 0 for num in data]
    return transformed_nums

average_bin, bin_0, bin_24 = get_diff_bin(data_46)
# print(len(average_bin), len(bin_0), len(bin_24))

diff_average_bin = transformer([(average_bin[i] - average_bin[i-1]) for i in range(1, len(average_bin))])
diff_bin_0_24 = [bin_0[i]- bin_24[i] for i in range(len(bin_24))][1:]
diff_bin_0_0 = [(bin_0[i]- bin_0[i-1])/bin_0[i-1] for i in range(1, len(bin_0))]
# print(len(diff_average_bin), len(diff_bin_0_0), len(diff_bin_0_24))

data_diff = pd.DataFrame({'ddiff_average_bin': diff_average_bin, 'bin_0_0': diff_bin_0_0, 'bin_0_24': diff_bin_0_24})
# data_diff.corr()

In [31]:
def df2matrix(file_path:str, col_name: str=None, bin_num: bool= True):
    """
    转换数据结构，以date为横坐标，制定col_name为纵坐标
    """

    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
    df.set_index('date', inplace= True)
    df.sort_index(inplace= True)
    n, k = df.shape[0], 25
    # TODO: 1、把 bin0 分离出来，而后补充到特征里面去，也就是需要在  self.genNewFeatureBinVolume() 把bin0 特征融合进来
    if bin_num:
        data = pd.DataFrame(np.array(df[col_name]).reshape(int(n/k), k),
                                columns= ['bin{}'.format(i) for i in range(k)]).drop('bin0', axis=1)
    else:
        bin0_list = []
        for i in range(0, df.shape[0], 25):
            if i == 0:
                bin0_list.append([(df[col_name].iloc[i]- df[col_name].iloc[i])/ df[col_name].iloc[i]])
            else:
                bin0_list.append([(df[col_name].iloc[i]- df[col_name].iloc[i-25])/ df[col_name].iloc[i-25]])
        result = np.array([[element[0]]*(k-1) for element in bin0_list])
        print(result.shape)
        data = pd.DataFrame(np.array(result),
                            columns= ['bin0' for i in range(k-1)])
    data['date'] = pd.to_datetime(list(sorted(set(df.index))), format='%Y/%m/%d')
    data.set_index('date', inplace= True)
    data.sort_index(inplace=True)
    return data

data = df2matrix(file_path= '../data/0308/000046_XSHE_25_daily.csv', col_name= 'bin_volume', bin_num= False)

(197, 24)


In [8]:
input_feature = np.random.rand(2, 4)
w = np.random.rand(4, 3)
a = np.random.rand(6, 2)
h = np.matmul(input_feature, w)
h1 = np.matmul(h, a[:3,:])
h2 = np.matmul(h, a[3:, :])
e = h1+ h2
e

array([[2.1871026 , 2.03988665],
       [1.67665943, 1.5774206 ]])